In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import graphviz 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

from frouros.detectors.concept_drift import DDM, DDMConfig
from frouros.metrics import PrequentialError

### Carregamento do conjunto de dados de todos os anos

In [2]:
enade_todos_anos = pd.read_csv("../../concept_drift/tabela_final_OHE_tres_anos_treinamento.csv")

# enade_todos_anos = pd.read_csv("../../tabela_BCC.csv")

In [3]:
numero_caracteristicas = enade_todos_anos.shape[1] - 1
    
X = enade_todos_anos.iloc[:, 0:numero_caracteristicas]
y = enade_todos_anos.iloc[:, -1]

In [4]:
X

,Numero_Notas_Invalidas,Numero_Faltantes,Numero_Participantes,ADS,BCC,EC,GTI,LCC,RC,SI,...,Não se aplica.42,nulos_Atendimento_Necessidades,Disc. totalmente.41,Discordo.41,Disc. parc..41,Concordo parc..41,Concordo.41,Concordo Total..41,Não sei responder.41,Não se aplica.43
0,0.0,0.250000,0.750000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.125000,0.062500,0.125000,0.000000,0.250000,0.250000,0.187500,0.000000,0.000000
1,0.0,0.142857,0.857143,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.035714,0.071429,0.214286,0.250000,0.107143,0.285714,0.071429,0.000000,0.000000,0.000000
2,0.0,0.232143,0.767857,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.035714,0.196429,0.071429,0.107143,0.160714,0.178571,0.125000,0.142857,0.000000,0.017857
3,0.0,0.216216,0.783784,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.162162,0.054054,0.108108,0.108108,0.108108,0.324324,0.135135,0.000000,0.000000
4,0.0,0.075000,0.925000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.025000,0.050000,0.000000,0.025000,0.000000,0.150000,0.200000,0.575000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4609,0.0,0.200000,0.800000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.200000,0.000000,0.000000,0.000000,0.100000,0.500000,0.200000,0.000000,0.000000
4610,0.0,0.266667,0.733333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.133333,0.200000,0.000000,0.000000,0.000000,0.266667,0.000000,0.466667,0.000000,0.066667
4611,0.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.250000,0.625000,0.000000,0.000000
4612,0.0,0.120000,0.880000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.040000,0.120000,0.040000,0.120000,0.080000,0.120000,0.200000,0.280000,0.000000,0.040000


In [4]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [5]:
X_res

,Numero_Notas_Invalidas,Numero_Faltantes,Numero_Participantes,ADS,BCC,EC,GTI,LCC,RC,SI,...,Muito pouco,Não contribui,Não sei responder.4,nulos_Formacao,Muito boa,Boa,Regular,Fraca,Muito fraca,Não sei responder.5
0,0.02439,0.097561,0.878049,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.170732,0.170732,0.048780,0.073171,0.121951,0.292683,0.341463,0.121951,0.024390,0.024390
1,0.00000,0.241071,0.758929,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.169643,0.133929,0.026786,0.098214,0.223214,0.250000,0.339286,0.044643,0.017857,0.026786
2,0.00000,0.171717,0.828283,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.060606,0.040404,0.000000,0.141414,0.282828,0.363636,0.202020,0.010101,0.000000,0.000000
3,0.00000,0.210526,0.789474,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.157895,0.017544,0.035088,0.000000,0.333333,0.368421,0.263158,0.035088,0.000000,0.000000
4,0.00000,0.085106,0.914894,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.021277,0.021277,0.000000,0.063830,0.446809,0.361702,0.127660,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905,0.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.104614,0.052307,0.000000,0.000000,0.521157,0.321922,0.156921,0.000000,0.000000,0.000000
11906,0.00000,0.077151,0.922849,0.000000,0.041502,0.958498,0.0,0.0,0.000000,0.0,...,0.124123,0.130043,0.020526,0.019561,0.279519,0.305000,0.312850,0.051798,0.019561,0.011711
11907,0.00000,0.210666,0.789334,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.063086,0.073600,0.000000,0.183575,0.490481,0.136686,0.147200,0.010514,0.021029,0.010514
11908,0.00000,0.268924,0.731076,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.0,...,0.066752,0.066752,0.000000,0.214948,0.339508,0.202172,0.161929,0.026509,0.000000,0.054934


In [7]:
y_res

0        3
1        4
2        4
3        4
4        4
        ..
11905    5
11906    5
11907    5
11908    5
11909    5
Name: Nota_Conceito_Faixa, Length: 11910, dtype: int64

In [5]:
df_smote = pd.concat([X_res, y_res], axis=1)

In [9]:
df_smote.to_csv(f'./df_smote.csv',index=False)

In [6]:
enade_todos_anos = pd.read_csv("./df_smote.csv")

### Cross-validation (k = 5)

In [3]:
k = 5

In [4]:
array_folds = np.array_split(enade_todos_anos, k)

In [6]:
# Detector configuration and instantiation
config = DDMConfig(
    warning_level=2.0,
    drift_level=3.0,
    min_num_instances=25,  # minimum number of instances before checking for concept drift
)
detector = DDM(config=config)

In [7]:
# Metric to compute accuracy
metric = PrequentialError(alpha=1.0)  # alpha=1.0 is equivalent to normal accuracy

In [8]:
drift_flag = False

In [5]:
for i_fold in range(k):
    
    # Separação dos folds
    
    folds_treinamento = array_folds.copy()
    fold_teste = array_folds[i_fold]
    
    del folds_treinamento[i_fold]
    
    folds_treinamento = pd.concat(folds_treinamento, sort=False)
    
    # Separação em X e y
    
    numero_caracteristicas = enade_todos_anos.shape[1] - 1
    
    X_folds_treinamento = folds_treinamento.iloc[:, 0:numero_caracteristicas]
    y_folds_treinamento = folds_treinamento.iloc[:, -1]
    
    X_fold_teste = fold_teste.iloc[:, 0:numero_caracteristicas]
    y_fold_teste = fold_teste.iloc[:, -1]
    
    # Normalização de treino e teste
    
    normalizador_treinamento = StandardScaler()
    
    normalizador_treinamento.fit(X_folds_treinamento)
    treinamento_normalizado = normalizador_treinamento.transform(X_folds_treinamento)
    
    normalizador_teste = StandardScaler()
    
    normalizador_teste.fit(X_fold_teste)
    teste_normalizado = normalizador_teste.transform(X_fold_teste)
    
    # Busca por hyperparâmetros
    
    parametros_para_busca = {"C": [0.01, 0.1, 1, 10, 100], 
              "kernel": ["linear", "poly", "rbf"],
              "gamma": [0.1, 1, 10],
              "tol": [0.1, 0.001, 0.00001]
             }
    
    classificador = SVC(probability=True)
    
    busca = HalvingGridSearchCV(classificador, parametros_para_busca, scoring="f1_weighted").fit(treinamento_normalizado, y_folds_treinamento)
    
    busca.best_estimator_.fit(treinamento_normalizado, y_folds_treinamento)
    y_predito = busca.best_estimator_.predict(teste_normalizado)
    
    #error = 1 - (y_predito.tolist() == y_fold_teste.tolist())
    #metric_error = metric(error_value=error)
    #_ = detector.update(value=error)
    #status = detector.status
    
    #if status["drift"] and not drift_flag:
        #drift_flag = True
        #print(f"Concept drift detected at step {i}. Accuracy: {1 - metric_error:.4f}")
    
    #print(f"Final accuracy: {1 - metric_error:.4f}\n")
    
    melhor_classificador = str(busca.best_estimator_)
    resultado = f1_score(y_fold_teste, y_predito, average='weighted')
    
    print(f"Teste: fold {i_fold}")
    print(f"Classificador: \n\n{melhor_classificador}\n")
    print(f"Resultado: {resultado}\n")
    
#if not drift_flag:
    #print("No concept drift detected")

Teste: fold 0
Classificador: 

SVC(C=0.01, gamma=0.1, kernel='linear', probability=True, tol=1e-05)

Resultado: 0.4591488779828817

Teste: fold 1
Classificador: 

SVC(C=0.01, gamma=10, kernel='linear', probability=True)

Resultado: 0.4545065807374534



KeyboardInterrupt: 

### Matriz de Confusão

In [ ]:
y_pred_HGS = search.predict(X_test)

In [ ]:
cm_HGS = confusion_matrix(y_test, y_pred_HGS, labels=search.classes_)
disp_HGS = ConfusionMatrixDisplay(confusion_matrix=cm_HGS,
                              display_labels=search.classes_)
disp_HGS.plot()
plt.show()